In [1]:
import cvxpy as cp
import numpy as np
np.set_printoptions(suppress=True, precision=2)

In [2]:
def gen_Q(Q_lists, N, p):
    Q_bar_lists = [[np.zeros((2*p, 2*p)) for _ in range(N)] for _ in range(N)]
    Q_bar_lists[N-1][N-1] = np.zeros((p, p))
    for i in range(N-1):
        Q_bar_lists[i][N-1] = np.zeros((2*p, p))
        Q_bar_lists[N-1][i] = np.zeros((p, 2*p))
    for i in range(N):
        for j in range(i, N):
            if j == i:
                if i < N-1:
                    Q_bar_lists[i][j] = np.block([
                        [-Q_lists[i][i] + Q_lists[i+1][i], Q_lists[i][i]],
                        [Q_lists[i][i], -Q_lists[i][i] - Q_lists[i+1][i]]
                    ])
                elif i == N - 1:
                    Q_bar_lists[i][j] = np.zeros((p, p))
            else: 
                if j < N - 1:
                    Q_bar_lists[i][j] = np.block([
                        [-Q_lists[j][i], Q_lists[j][i]],
                        [Q_lists[j][i], -Q_lists[j][i]]
                    ])
                elif j == N - 1:
                    Q_bar_lists[i][j] = np.block([
                        [-Q_lists[j][i]],
                        [Q_lists[j][i]]
                    ])
    for i in range(N):
        for j in range(i):
            Q_bar_lists[i][j] = Q_bar_lists[j][i].T

    Q = np.block([
        [Q_bar_lists[i][j] for j in range(N)] for i in range(N)
    ])

    return Q

In [3]:
q_self = 1
q_pred = 1
N = 3
p = 2

Q_lists_pf =[[] for _ in range(N)]
for i in range(N):
    for j in range(N):
        if i == j and i < N-1:
            Q_lists_pf[i].append(q_self * np.eye(p))
        elif j == i-1:
            Q_lists_pf[i].append(q_pred * np.eye(p))
        else: Q_lists_pf[i].append(np.zeros((p, p)))

Q_pf = gen_Q(Q_lists_pf, N, p)

# for i in range(N):
#     for j in range(len(Q_lists_pf[i])):
#         print(f"Q({i},{j}) = ")
#         print(Q_lists_pf[i][j])
#         print()

print(Q_pf)

[[ 0.  0.  1.  0. -1. -0.  1.  0. -0. -0.]
 [ 0.  0.  0.  1. -0. -1.  0.  1. -0. -0.]
 [ 1.  0. -2. -0.  1.  0. -1. -0.  0.  0.]
 [ 0.  1. -0. -2.  0.  1. -0. -1.  0.  0.]
 [-1. -0.  1.  0.  0.  0.  1.  0. -1. -0.]
 [-0. -1.  0.  1.  0.  0.  0.  1. -0. -1.]
 [ 1.  0. -1. -0.  1.  0. -2. -0.  1.  0.]
 [ 0.  1. -0. -1.  0.  1. -0. -2.  0.  1.]
 [-0. -0.  0.  0. -1. -0.  1.  0.  0.  0.]
 [-0. -0.  0.  0. -0. -1.  0.  1.  0.  0.]]


In [4]:
print(f"trace of Q: {np.trace(Q_pf)}")
eigvals, eigvecs = np.linalg.eig(Q_pf)
eigvals, eigvecs = np.real(eigvals), np.real(eigvecs)
# print(np.round(eigvecs.T @ eigvecs, 3))
print(f"Q is symmetric: {np.isclose(Q_pf, Q_pf.T).all()}")

eigvecs[np.isclose(eigvecs, 0, rtol=0.0, atol=1e-5)] = 0

print("\neigenvalues:")
print(np.round(np.real(eigvals), 4))

pos_eigvals = eigvals[eigvals > 0]
pos_eigvecs = np.round(eigvecs[:, eigvals > 0], 3)
neg_eigvals = eigvals[eigvals < 0]
neg_eigvecs = np.round(eigvecs[:, eigvals < 0], 3)

print(f"\nsum positive eigenvals: {np.sum(pos_eigvals)}")
print("positive eigvals/vecs:")
pos_str = "  "
for i in range(len(pos_eigvals)):
    nnz_inds = np.nonzero(pos_eigvecs[:, i])
    min_mag = np.min(np.abs(pos_eigvecs[nnz_inds, i]))
    pos_eigvecs[:, i] /= min_mag
    pos_str += f"{pos_eigvals[i]:.4f} "
print(f"{pos_str}")
print(f"{np.round(pos_eigvecs, 2)}")

print(f"\nsum negative eigenvals: {np.sum(neg_eigvals)}")
print("negative eigvals/vecs:")
neg_str = "  "
for i in range(len(neg_eigvals)):
    nnz_inds = np.nonzero(neg_eigvecs[:, i])
    min_mag = np.min(np.abs(neg_eigvecs[nnz_inds, i]))
    neg_eigvecs[:, i] /= min_mag
    neg_str += f"{neg_eigvals[i]:.4f} "
print(f"{neg_str}")
print(f"{np.round(neg_eigvecs, 2)}")

trace of Q: -8.0
Q is symmetric: True

eigenvalues:
[ 1.47  0.34 -4.46 -4.46 -1.35 -1.35  1.47  0.34  0.   -0.  ]

sum positive eigenvals: 3.6203861703148528
positive eigvals/vecs:
  1.4715 0.3387 1.4715 0.3387 0.0000 
[[ -5.22   6.6    5.29   8.31   0.  ]
 [  0.     0.    19.29  -6.64   0.  ]
 [  1.     3.26  -1.     4.11   0.  ]
 [  0.     0.    -3.67  -3.28   0.  ]
 [  7.29   2.02  -7.37   2.55  -1.  ]
 [  0.     0.   -26.96  -2.04  -3.64]
 [ -1.39   1.     1.42   1.25  -1.  ]
 [  0.     0.     5.12  -1.    -3.64]
 [ -5.9   -3.02   5.96  -3.81  -1.  ]
 [  0.     0.    21.79   3.04  -3.64]]

sum negative eigenvals: -11.62038617031485
negative eigvals/vecs:
  -4.4609 -4.4609 -1.3493 -1.3493 -0.0000 
[[ -1.55   1.55   1.43  -1.43   0.  ]
 [  0.    17.3    0.    -7.64   0.  ]
 [  2.44  -2.45  -9.63   9.79   0.  ]
 [  0.   -27.3    0.    51.36   0.  ]
 [ -1.73   1.75  -1.     1.    -1.35]
 [  0.    19.35   0.     5.29   1.  ]
 [  2.73  -2.75   6.7   -6.79  -1.35]
 [  0.   -30.55   0.   -

In [5]:
Q = cp.Variable((2*N*p - p, 2*N*p - p), symmetric=True, name="Q")
cost = cp.trace(Q)
constraints = [Q << 0, cp.trace(Q) >= -100]

# Q_self = cp.Variable((p, p), name="Q_self")
# constraints += [Q_self == np.eye(p)]
# Q_pred = cp.Variable((p, p), name="Q_pred")
# constraints += [Q_pred >> 0]

Q_lists = [[cp.Variable((p, p), symmetric=True, name=f"Q_{i+1},{j+1}") for j in range(N)] for i in range(N)]
for i in range(N):
    for j in range(N):
        if i == j and i < N-1:
            constraints += [Q_lists[i][j] >> 0, Q_lists[i][j] << 10*np.eye(p)]  # == Q_self]
        elif j == i-1:
            constraints += [Q_lists[i][j] >> 0]  # == Q_pred]
        elif j == N-1: 
            constraints += [Q_lists[i][j] == np.zeros((p, p))]
# constraints += [Q_lists[0][0] == np.eye(p)]

Q_bar_lists = [[] for _ in range(N)]
for i in range(N-1):
    for j in range(N-1):
        Q_bar_lists[i].append(cp.Variable((2*p, 2*p), symmetric=True, name=f"Q_bar_{i+1},{j+1}"))
    Q_bar_lists[i].append(cp.Variable((2*p, p), name=f"Q_bar_{i+1},{N}"))
for j in range(N-1):
    Q_bar_lists[N-1].append(cp.Variable((p, 2*p), name=f"Q_bar_{N},{j+1}"))
Q_bar_lists[N-1].append(cp.Variable((p, p), name=f"Q_bar_{N},{N}"))

for i in range(N):
    for j in range(i, N):
        if j == i:
            if i < N-1:
                constraints += [Q_bar_lists[i][j] == cp.bmat([
                    [-Q_lists[i][i] + Q_lists[i+1][i], Q_lists[i][i]],
                    [Q_lists[i][i], -Q_lists[i][i] - Q_lists[i+1][i]]
                ])]
            elif i == N - 1:
                constraints += [Q_bar_lists[i][j] == np.zeros((p, p))]
        else: 
            if j < N - 1:
                constraints += [Q_bar_lists[i][j] == cp.bmat([
                    [-Q_lists[j][i], Q_lists[j][i]],
                    [Q_lists[j][i], -Q_lists[j][i]]
                ])]
            elif j == N - 1:
                constraints += [Q_bar_lists[i][j] == cp.bmat([
                    [-Q_lists[j][i]],
                    [Q_lists[j][i]]
                ])]

constraints += [Q == cp.bmat([
    [Q_bar_lists[i][j] for j in range(N)] for i in range(N)
])]
# for constraint in constraints:
#     print(constraint)

prob = cp.Problem(cp.Minimize(cost), constraints)
prob.solve()
print(prob.status)
print(Q.value)
print(np.linalg.eigvals(Q.value))
for i in range(1, N+1):
    for j in range(1, N+1):
        if i == j or j == i-1:
            print(f'Q_{i},{j}:')
            print(prob.var_dict[f'Q_{i},{j}'].value)

optimal
[[-10.   0.  10.  -0.  -0.  -0.   0.   0.   0.   0.]
 [  0. -10.  -0.  10.  -0.  -0.   0.   0.   0.   0.]
 [ 10.  -0. -10.  -0.   0.   0.  -0.  -0.  -0.  -0.]
 [ -0.  10.  -0. -10.   0.   0.  -0.  -0.  -0.  -0.]
 [ -0.  -0.   0.   0. -10.   0.  10.  -0.  -0.  -0.]
 [ -0.  -0.   0.   0.   0. -10.  -0.  10.  -0.  -0.]
 [  0.   0.  -0.  -0.  10.  -0. -10.  -0.   0.   0.]
 [  0.   0.  -0.  -0.  -0.  10.  -0. -10.   0.   0.]
 [  0.   0.  -0.  -0.  -0.  -0.   0.   0.  -0.  -0.]
 [  0.   0.  -0.  -0.  -0.  -0.   0.   0.  -0.  -0.]]
[-20.+0.j -20.+0.j -20.+0.j -20.+0.j  -0.+0.j  -0.-0.j   0.+0.j   0.+0.j
   0.+0.j   0.+0.j]
Q_1,1:
[[10. -0.]
 [-0. 10.]]
Q_2,1:
[[0. 0.]
 [0. 0.]]
Q_2,2:
[[10.  0.]
 [ 0. 10.]]
Q_3,2:
[[0. 0.]
 [0. 0.]]
Q_3,3:
[[0. 0.]
 [0. 0.]]
